In [16]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jan  4 19:34:45 2023

@author
"""

import numpy as np
import itertools
import random
import matplotlib.pyplot as plt
import math
from matplotlib.colors import ListedColormap
from matplotlib.cm import hsv


def generate_colormap(number_of_distinct_colors: int = 80):
    ''' 
        Source - https://stackoverflow.com/questions/42697933/colormap-with-maximum-distinguishable-colours
        Response by Greg0ry
        Original code by xuancong84
    '''
    if number_of_distinct_colors == 0:
        number_of_distinct_colors = 80

    number_of_shades = 7
    number_of_distinct_colors_with_multiply_of_shades = int(math.ceil(number_of_distinct_colors / number_of_shades) * number_of_shades)

    # Create an array with uniformly drawn floats taken from <0, 1) partition
    linearly_distributed_nums = np.arange(number_of_distinct_colors_with_multiply_of_shades) / number_of_distinct_colors_with_multiply_of_shades

    # We are going to reorganise monotonically growing numbers in such way that there will be single array with saw-like pattern
    #     but each saw tooth is slightly higher than the one before
    # First divide linearly_distributed_nums into number_of_shades sub-arrays containing linearly distributed numbers
    arr_by_shade_rows = linearly_distributed_nums.reshape(number_of_shades, number_of_distinct_colors_with_multiply_of_shades // number_of_shades)

    # Transpose the above matrix (columns become rows) - as a result each row contains saw tooth with values slightly higher than row above
    arr_by_shade_columns = arr_by_shade_rows.T

    # Keep number of saw teeth for later
    number_of_partitions = arr_by_shade_columns.shape[0]

    # Flatten the above matrix - join each row into single array
    nums_distributed_like_rising_saw = arr_by_shade_columns.reshape(-1)

    # HSV colour map is cyclic (https://matplotlib.org/tutorials/colors/colormaps.html#cyclic), we'll use this property
    initial_cm = hsv(nums_distributed_like_rising_saw)

    lower_partitions_half = number_of_partitions // 2
    upper_partitions_half = number_of_partitions - lower_partitions_half

    # Modify lower half in such way that colours towards beginning of partition are darker
    # First colours are affected more, colours closer to the middle are affected less
    lower_half = lower_partitions_half * number_of_shades
    for i in range(3):
        initial_cm[0:lower_half, i] *= np.arange(0.2, 1, 0.8/lower_half)

    # Modify second half in such way that colours towards end of partition are less intense and brighter
    # Colours closer to the middle are affected less, colours closer to the end are affected more
    for i in range(3):
        for j in range(upper_partitions_half):
            modifier = np.ones(number_of_shades) - initial_cm[lower_half + j * number_of_shades: lower_half + (j + 1) * number_of_shades, i]
            modifier = j * modifier / upper_partitions_half
            initial_cm[lower_half + j * number_of_shades: lower_half + (j + 1) * number_of_shades, i] += modifier

    return ListedColormap(initial_cm)

class BlottoGame:

    def generateActions(self,S,N):
    	indexCombs = list(itertools.combinations([i for i in range(N+S-1)], N-1))
    	fixedIndex = N+S-1
    	actionsList = []
    	for i in indexCombs:
    		action = []
    		memo = -1
    		for j in i:
    			action += [j-memo-1]
    			memo = j
    		action += [fixedIndex-memo-1]
    		actionsList += [tuple(action)]
    		
    	return actionsList

    
    def __init__(self,S,N):
        self.iteration = 1
        self.actionsList = np.array(self.generateActions(S, N))
        gameMatrix = []
        cmp = lambda x,y: float(x > y) if x >= y else -1.0
        for i in self.actionsList:
            rowPlayer = []
            for j in self.actionsList:
                rowPlayer += [sum([cmp(*k) for k in zip(i,j)])]

            gameMatrix+= [rowPlayer]
        self.gameMatrix = np.array(gameMatrix,int)
        
        self.p1Strat = np.array([1.0/len(self.actionsList) for x in self.actionsList])
        self.p2Strat = np.array([1.0/len(self.actionsList) for x in self.actionsList])
        # Define p(strategy) to be 1/number of strategies initially (uniform)
        
        
        self.p1StratTotal = np.array([1.0/len(self.actionsList) for x in self.actionsList])
        self.p2StratTotal = np.array([1.0/len(self.actionsList) for x in self.actionsList])
        # Strategy total is equal to just the first strategy
        
        self.iterationStrats = [(self.p1StratTotal,self.p2StratTotal)]
        # List holding every average strategy
        
        self.p1Regrets = np.array([0.0 for x in self.actionsList])
        self.p2Regrets = np.array([0.0 for x in self.actionsList])
        # Cumulative regrets are initially zero 
        
        
        self.regretsMapper = np.vectorize(lambda x: x if x > 0 else 0.0)
        # if a regret is negative just make it 0 
        
        self.iteratedExpected = []
 
    def chooseStrategy(self):
        i = 0
        cumulativeP1 = self.p1Strat[0]
        cumulativeP2 = self.p2Strat[0]
        p1Rand = random.random()
        p2Rand = random.random()

        while cumulativeP1 <= p1Rand:
            i += 1
            cumulativeP1 += self.p1Strat[i]
            
        j = 0

        while cumulativeP2 <= p2Rand:
            j+=1
            cumulativeP2 += self.p2Strat[j]
            
        return (i,j)
    
    def iterate(self):
        row,col = self.chooseStrategy()
        result = self.gameMatrix[row,col]
        newRowRegrets = (self.gameMatrix[:,col]- np.full(self.gameMatrix.shape[0],result) )

        newColRegrets = -1 * (self.gameMatrix[row,:] - np.full(self.gameMatrix.shape[1], result) )

        # Regrets is result - alternativeresult_i for all i. For column, negative as trying to minimise 'payoff'
        
        self.p1Regrets = self.p1Regrets + newRowRegrets
        self.p2Regrets = self.p2Regrets + newColRegrets
        # Accumulate new regrets 
        
        p1PositiveRegrets = self.regretsMapper(self.p1Regrets)
        p2PositiveRegrets = self.regretsMapper(self.p2Regrets)
        # Map the regrets to positive for strategy calculation 
        
        p1rSum = np.sum(p1PositiveRegrets) 
        p2rSum = np.sum(p2PositiveRegrets) 
        # Sum the positive regrets

        if p1rSum > 0:
            self.p1Strat = p1PositiveRegrets / p1rSum
        else:
            self.p1Strat = np.full(self.gameMatrix.shape[0],1.0/self.gameMatrix.shape[0])
        if p2rSum > 0:
            self.p2Strat = p2PositiveRegrets / p2rSum
        else:
            self.p2Strat = np.full(self.gameMatrix.shape[1],1.0/self.gameMatrix.shape[1])
        # If we have any positive regrets, we allocate probabilities in the new
        # strategy proportional to positive regret
        # Otherwise we distribute uniformally across possible strategies
            
        self.p1StratTotal = self.p1StratTotal + self.p1Strat
        self.p2StratTotal = self.p2StratTotal + self.p2Strat
        # We sum the newly calculated strategy such that we can work out the new
        # average strategy
        
        self.iteration += 1 
        # our iteration counter is necessary for working out averages as well
        # as indexing our findings
        
        self.iterationStrats += [(self.p1StratTotal / float(self.iteration), self.p2StratTotal / float(self.iteration) ) ] 
        # We record the average strategy after every iteration so that we can
        # See how the strategies evolve
        
        
    def expectedValue(self):
        self.iteratedExpected += [np.dot(self.iterationStrats[-1][0],np.matmul(self.gameMatrix,self.iterationStrats[-1][1]))]
        
        
    def train(self, limit):
        for i in range(limit):
            if i%100 == 0: print(i)
            self.iterate()
            self.expectedValue()
        
        
'''
'''        

limit = 1000
while limit < 10001:
    a = BlottoGame(5, 3)
    a.train(limit)
    limit *= 2
    mat = np.matmul(np.array([a.iterationStrats[-1][0]]).T,np.array([a.iterationStrats[-1][1]]))

    
    plt.imshow(mat, cmap='hot', interpolation='nearest',origin='lower')
    plt.colorbar()
    plt.title('Heatmap showing the joint probability distribution for the game')
    plt.savefig('Heatmap_'+str(a.iteration)+'.png')
    plt.close('all')
    plt.plot(a.iteratedExpected)
    plt.title('Expected value of the game for each iterations strategy')
    plt.savefig('expectedValue_'+str(a.iteration)+'.png')
    plt.close('all')

    plt.figure(figsize = (16,10))    
    plt.stackplot([i for i in range(len(a.iterationStrats))],np.array([x[0] for x in a.iterationStrats]).T,colors = generate_colormap(len(a.actionsList)).colors)
    plt.legend(['('+', '.join([str(i) for i in x])+')' for x in a.actionsList])

    plt.title('Probability for each strategy over each iteration for player A')
    plt.savefig('stackplot_'+str(a.iteration)+'.png')
    plt.close('all')
    
    b = sorted(list(zip(a.iterationStrats[-1][0],[tuple(a.actionsList[i]) for i in range(len(a.iterationStrats[-1][0]))])))
    plt.figure(figsize=(40,16))
    plt.bar([str(x[1]) for x in b], [x[0] for x in b])
    plt.savefig('test_'+str(a.iteration)+'.png')
    plt.close('all')
    
    

0
100
200
300
400
500
600
700
800
900
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900


In [1]:
c_final=np.array([avg_c_a[i]/i for i in range (1,iterations)])
print(np.shape(c_final))

NameError: name 'np' is not defined

In [17]:
a.iterationStrats[-1][0]

array([1.78549110e-05, 1.66512268e-03, 1.02651036e-01, 1.29557106e-01,
       1.20766715e-04, 1.78549110e-05, 1.53886430e-03, 1.10031893e-01,
       6.64886261e-04, 9.18406121e-02, 5.58217489e-04, 1.02783883e-01,
       5.27117577e-04, 4.40697152e-05, 1.11904814e-01, 1.29667798e-01,
       8.91172662e-02, 1.25646834e-01, 4.90419802e-04, 1.13572838e-03,
       1.78549110e-05])